In [61]:
# 检测环境是否安装
%time !pip install pandas openpyxl
%matplotlib inline

Wall time: 1.29 s


In [62]:
import pandas as pd
dataset = pd.read_excel('dataset/regression_train.xlsx')
test_dataset = pd.read_excel('dataset/regression_test.xlsx')
## pre=processing
# delete when the PVID exists more than one time
dataset.drop_duplicates(subset=['PVID'],keep=False)
test_dataset.drop_duplicates(subset=['PVID'],keep=False)
# delete the nan lines
dataset = dataset.dropna(axis=0,how='any')
# delete the nan lines
test_dataset = test_dataset.dropna(axis=0,how='any')
# check whether the age is normal, clear the data with age == 91.4
dataset['age'].unique()
dataset=dataset.sort_values(by=['age'])
dataset=dataset[dataset.age<90]

test_dataset = test_dataset[test_dataset.age < 90]

# check the gender
dataset['gender'].unique()
# check the HR
dataset = dataset.sort_values(by=['HR'])
dataset['HR'].unique()

# check the RR
dataset = dataset.sort_values(by=['RR'])
dataset['RR'].unique()

# check the SBP
dataset = dataset.sort_values(by=['SBP'])
dataset['SBP'].unique()

# check the SPO2
dataset = dataset.sort_values(by=['SPO2'])
dataset['SPO2'].unique()

#check the DBP
dataset = dataset.sort_values(by=['DBP'])
dataset['DBP'].unique()     

#check the MAP
dataset = dataset.sort_values(by=['MAP'])
dataset['MAP'].unique()

#sort by id
dataset = dataset.sort_values(by=['PVID'])

dataset

,PVID,age,gender,HR,RR,SBP,SPO2,DBP,MAP,PO2
0,1,65.0,1,68.0,16.0,101.0,94.0,50.0,61.0,79
1,2,65.0,1,76.0,17.0,131.0,94.0,53.0,72.0,72
2,3,65.0,1,87.0,16.0,114.0,96.0,41.0,60.0,92
3,4,69.0,1,135.0,13.0,107.0,99.0,71.0,84.0,40
4,5,72.0,0,80.0,18.0,122.0,95.0,29.0,63.0,82
...,...,...,...,...,...,...,...,...,...,...
4815,4816,85.0,1,88.0,20.0,102.0,95.0,58.0,72.0,36
4816,4817,71.0,0,70.0,28.0,100.0,100.0,61.0,75.0,142
4817,4818,71.0,0,74.0,30.0,87.0,100.0,58.0,68.0,259
4818,4819,71.0,0,63.0,32.0,180.0,100.0,82.0,112.0,192


In [63]:
%time !pip install scikit-learn
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

# scale the age 
age_scale_param = scaler.fit(dataset['age'].values.reshape(-1, 1))
dataset['age_scaled']=scaler.fit_transform(dataset['age'].values.reshape(-1, 1), age_scale_param)
test_dataset['age_scaled']=scaler.fit_transform(test_dataset['age'].values.reshape(-1, 1), age_scale_param)
# scale the HR
HR_scale_param = scaler.fit(dataset['HR'].values.reshape(-1, 1))
dataset['HR_scaled']=scaler.fit_transform(dataset['HR'].values.reshape(-1, 1), HR_scale_param)
test_dataset['HR_scaled']=scaler.fit_transform(test_dataset['HR'].values.reshape(-1, 1), HR_scale_param)

# scale the RR
RR_scale_param = scaler.fit(dataset['RR'].values.reshape(-1, 1))
dataset['RR_scaled']=scaler.fit_transform(dataset['RR'].values.reshape(-1, 1), RR_scale_param)
test_dataset['RR_scaled']=scaler.fit_transform(test_dataset['RR'].values.reshape(-1, 1), RR_scale_param)
# scale the SBP
SBP_scale_param = scaler.fit(dataset['SBP'].values.reshape(-1, 1))
dataset['SBP_scaled']=scaler.fit_transform(dataset['SBP'].values.reshape(-1, 1), SBP_scale_param)
test_dataset['SBP_scaled']=scaler.fit_transform(test_dataset['SBP'].values.reshape(-1, 1), SBP_scale_param)
# scale the SPO2
SPO2_scale_param = scaler.fit(dataset['SPO2'].values.reshape(-1, 1))
dataset['SPO2_scaled']=scaler.fit_transform(dataset['SPO2'].values.reshape(-1, 1), SPO2_scale_param)
test_dataset['SPO2_scaled']=scaler.fit_transform(test_dataset['SPO2'].values.reshape(-1, 1), SPO2_scale_param)
# scale the DBP
DBP_scale_param = scaler.fit(dataset['DBP'].values.reshape(-1, 1))
dataset['DBP_scaled']=scaler.fit_transform(dataset['DBP'].values.reshape(-1, 1), DBP_scale_param)
test_dataset['DBP_scaled']=scaler.fit_transform(test_dataset['DBP'].values.reshape(-1, 1), DBP_scale_param)
#scale the MAP
MAP_scale_param = scaler.fit(dataset['MAP'].values.reshape(-1, 1))
dataset['MAP_scaled']=scaler.fit_transform(dataset['MAP'].values.reshape(-1, 1), MAP_scale_param)
test_dataset['MAP_scaled']=scaler.fit_transform(test_dataset['MAP'].values.reshape(-1, 1), MAP_scale_param)
# find the corr factors
corr_matrix = dataset.corr()

corr_matrix['PO2'].sort_values(ascending=False)

Wall time: 1.29 s


PO2            1.000000
SPO2_scaled    0.310284
SPO2           0.310284
MAP            0.041555
MAP_scaled     0.041555
SBP            0.021784
SBP_scaled     0.021784
gender         0.014617
DBP_scaled    -0.009623
DBP           -0.009623
PVID          -0.016509
age           -0.029970
age_scaled    -0.029970
HR            -0.061321
HR_scaled     -0.061321
RR            -0.128123
RR_scaled     -0.128123
Name: PO2, dtype: float64

In [64]:
from sklearn import metrics

from sklearn import linear_model

features = ['SPO2_scaled','MAP_scaled','SBP_scaled','age_scaled','gender','HR_scaled','RR_scaled']


In [65]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from numpy import mean
kfold_num=15
kf= KFold(kfold_num,shuffle=True,random_state = 2)

# linear regression
print('-----------------linear regression---------------------')
mse = []
r2_scores = []
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    linearRegressionModel = linear_model.LinearRegression()
    linearRegressionModel.fit(train_x,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    predict_valid_y = linearRegressionModel.predict(valid_x)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))

# Polynomial regression 
print('-----------------polynomial linear regression---------------------')
mse = []
r2_scores = []
from sklearn.preprocessing import PolynomialFeatures
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    poly_reg = PolynomialFeatures(degree=2)
    train_x_poly = poly_reg.fit_transform(train_x)
    linearRegressionModel = linear_model.LinearRegression()
    linearRegressionModel.fit(train_x_poly,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    valid_x_poly = poly_reg.fit_transform(valid_x)
    predict_valid_y = linearRegressionModel.predict(valid_x_poly)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))


# KNN neighbors regressor
print('-----------------knn neighbors regression---------------------')
mse = []
r2_scores = []
from sklearn.neighbors import KNeighborsRegressor
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    model = KNeighborsRegressor(n_neighbors=4)
    model.fit(train_x,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    predict_valid_y = model.predict(valid_x)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))
    

-----------------linear regression---------------------
the mean squared error: 2403.0752
the r2 score: -7.7610
-----------------polynomial linear regression---------------------
the mean squared error: 2302.8886
the r2 score: -3.9246
-----------------knn neighbors regression---------------------
the mean squared error: 2735.7465
the r2 score: -1.8245


In [66]:
 # Decision Tree Regression
print('-----------------decision tree regression---------------------')
mse = []
r2_scores = []
from sklearn.tree import DecisionTreeRegressor
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    model = DecisionTreeRegressor(max_depth=3)
    model.fit(train_x,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    predict_valid_y = model.predict(valid_x)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))
    

-----------------decision tree regression---------------------
the mean squared error: 2247.3271
the r2 score: -3.5903


In [67]:
# ada boost regression
from sklearn.ensemble import AdaBoostRegressor
mse = []
r2_scores = []
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    model = AdaBoostRegressor(n_estimators=2)
    model.fit(train_x,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    predict_valid_y = model.predict(valid_x)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))
    

the mean squared error: 2260.3212
the r2 score: -3.2012


In [68]:
# random forest regreesion
from sklearn.ensemble import RandomForestRegressor
mse = []
r2_scores = []
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    model = RandomForestRegressor(n_estimators=50,random_state=0)
    model.fit(train_x,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    predict_valid_y = model.predict(valid_x)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))

the mean squared error: 2416.7210
the r2 score: -2.0523


In [69]:
# svm regression 
from sklearn.svm import SVR
mse = []
r2_scores = []
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    model = SVR(kernel='linear', gamma=0.01, C=1, epsilon=0.2)
    model.fit(train_x,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    predict_valid_y = model.predict(valid_x)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))

the mean squared error: 2469.2008
the r2 score: -6.3500


In [70]:
# MLP regression

from sklearn.neural_network import MLPRegressor
mse = []
r2_scores = []
for train,valid in kf.split(dataset):
    train_x=dataset[features].iloc[train,:]
    train_y = dataset['PO2'].iloc[train]
    model = MLPRegressor(max_iter=10000)
    model.fit(train_x,train_y)
    valid_x = dataset[features].iloc[valid,:]
    valid_y = dataset['PO2'].iloc[valid]
    predict_valid_y = model.predict(valid_x)
    mse.append(mean_squared_error(predict_valid_y,valid_y))
    # print("the mean squared error: %.4f" % mse)
    r2_scores.append(r2_score(predict_valid_y,valid_y))
    # print("the r2 score: %.4f" % mse)
print("the mean squared error: %.4f" % mean(mse))
print("the r2 score: %.4f" % mean(r2_scores))

the mean squared error: 2215.4417
the r2 score: -3.0546
